In [15]:
import sys
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from IPython.display import HTML
from IPython.display import Markdown as md
from IPython.core.display import HTML as Center
from utilities import *
from config import marker, start_date, end_date, demographics, codelist_code_column, codelist_term_column, codelist_path


%matplotlib inline


Center('''<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>''')

class Measure:
  def __init__(self, id, numerator, denominator, group_by):
    self.id = id
    self.numerator = numerator
    self.denominator = denominator
    self.group_by = group_by
    
measures = [

    Measure(
        id="total",
        numerator="event",
        denominator="population",
        group_by=["age_band"]
    ),

    Measure(
        id="event_code",
        numerator="event",
        denominator="population",
        group_by=["age_band","event_code"]
    ),

    Measure(
        id="practice",
        numerator="event",
        denominator="population",
        group_by=["age_band","practice"]
    ),


]

for d in demographics:
    if d=='age_band':
        m = Measure(
        id=d,
        numerator="event",
        denominator="population",
        group_by=["age_band"]
        )
        measures.append(m)
    else:
        m = Measure(
            id=d,
            numerator="event",
            denominator="population",
            group_by=["age_band", d]
        )
        measures.append(m)

default_measures = ['total', 'event_code', 'practice']
measures_ids = default_measures+ demographics
measures_dict = {}

for m in measures:
    measures_dict[m.id] = m






In [16]:
display(
md("# Service Restoration Observatory"),
md(f"## Changes in {marker} between {start_date} and {end_date}"),
md(f"Below are various time-series graphs showing changes in {marker} code use."),
)


# Service Restoration Observatory

## Changes in ACE Inhibitors between 2021-06-01 and 2021-12-01

Below are various time-series graphs showing changes in ACE Inhibitors code use.

In [17]:
display(
md("### Methods"),
md(f"Using OpenSAFELY-TPP, covering 40% of England's population, we have assessed coding activity related to {marker} between {start_date} and {end_date}. The codelist used can be found here at [OpenSAFELY Codelists](https://codelists.opensafely.org/).  For each month within the study period, we have calculated the rate at which the code was recorded per 1000 registered patients."),
md(f"All analytical code and output is available for inspection at the [OpenSAFELY GitHub repository](https://github.com/opensafely")
)


### Methods

Using OpenSAFELY-TPP, covering 40% of England's population, we have assessed coding activity related to ACE Inhibitors between 2021-06-01 and 2021-12-01. The codelist used can be found here at [OpenSAFELY Codelists](https://codelists.opensafely.org/).  For each month within the study period, we have calculated the rate at which the code was recorded per 1000 registered patients.

All analytical code and output is available for inspection at the [OpenSAFELY GitHub repository](https://github.com/opensafely

In [18]:
default_measures = ['total', 'event_code', 'practice']
measures = default_measures+ demographics

data_dict = {}

for key, value in measures_dict.items():
    
    df = pd.read_csv(f'../output/measure_{value.id}.csv')
    if key == "event_code":
        df.round(16)
    
    to_datetime_sort(df)
    
    if value.id=='age_band':
        data_dict[value.id] = calculate_rate(df, m=value, rate_per=1000, return_age=True)
    elif key == "imd":
       
        df = calculate_rate(df, m=value, rate_per=1000)
       
        df_grouped = calculate_imd_group(df, 'event', 'rate_standardised')
        
        data_dict[value.id] = df_grouped
    
    elif key == "ethnicity":
        df = convert_ethnicity(df)
        data_dict[value.id] = calculate_rate(df, m=value, rate_per=1000)

    else:
        data_dict[value.id] = calculate_rate(df, m=value, rate_per=1000)




codelist = pd.read_csv(f'../{codelist_path}')



In [19]:
display(
md(f"## Total {marker} Number")
)


## Total ACE Inhibitors Number

In [20]:
plot_measures(data_dict['total'], title=f"Total {marker} across whole population", column_to_plot='rate', category=False, y_label='Rate per 1000')


In [21]:
display(
md("### Sub totals by sub codes"),
md("Events for the top 5 subcodes across the study period"))
child_table = create_child_table(df=data_dict['event_code'], code_df=codelist, code_column=codelist_code_column, term_column=codelist_term_column)
child_table
    

### Sub totals by sub codes

Events for the top 5 subcodes across the study period

,CTV3Code,Events,Events (thousands),Description
G573.,G573.,3535.0,3.535,Atrial fibrillation and flutter


In [22]:
display(
md("## Total Number by GP Practice")
)


## Total Number by GP Practice

In [23]:

practices_dict =calculate_statistics_practices(data_dict['practice'], practice_df,end_date)
print(f'Practices included entire period: {practices_dict["total"]["number"]} ({practices_dict["total"]["percent"]}%)')
print(f'Practices included within last year: {practices_dict["year"]["number"]} ({practices_dict["year"]["percent"]}%)')
print(f'Practices included within last 3 months: {practices_dict["months_3"]["number"]} ({practices_dict["months_3"]["percent"]}%)')
interactive_deciles_chart(data_dict['practice'], period_column='date', column='event', title='Decile chart',ylabel='rate per 1000')


Practices included entire period: 44 (104.76%)
Practices included within last year: 44 (104.76%)
Practices included within last 3 months: 42 (100.0%)


In [24]:
i=0


In [25]:
    display(
    md(f"## Breakdown by {demographics[i]}")
    )
    counts_df = calculate_statistics_demographics(df=data_dict[demographics[i]], demographic_var=demographics[i], end_date=end_date, event_column='event')
    counts_df
    

## Breakdown by sex

,Total Study Period,Within Last Year,Within Last 3 Months
F,17445.0,17445.0,7501.0
M,17196.0,17196.0,7342.0
U,359.0,359.0,157.0


In [26]:
    plot_measures(data_dict[demographics[i]], title=f'Breakdown by {demographics[i]}', column_to_plot='rate_standardised', category=demographics[i], y_label='Standardised Rate per 1000')
    i+=1
    

In [27]:
    display(
    md(f"## Breakdown by {demographics[i]}")
    )
    counts_df = calculate_statistics_demographics(df=data_dict[demographics[i]], demographic_var=demographics[i], end_date=end_date, event_column='event')
    counts_df
    

## Breakdown by age_band

,Total Study Period,Within Last Year,Within Last 3 Months
0-4,1785.0,1785.0,751.0
10-14,1744.0,1744.0,743.0
15-19,1714.0,1714.0,704.0
20-24,1790.0,1790.0,773.0
25-29,1708.0,1708.0,725.0
30-34,1755.0,1755.0,766.0
35-39,1803.0,1803.0,762.0
40-44,1794.0,1794.0,789.0
45-49,3571.0,3571.0,1554.0
5-9,1668.0,1668.0,733.0


In [28]:
    plot_measures(data_dict[demographics[i]], title=f'Breakdown by {demographics[i]}', column_to_plot='rate_standardised', category=demographics[i], y_label='Standardised Rate per 1000')
    i+=1
    

In [29]:
    display(
    md(f"## Breakdown by {demographics[i]}")
    )
    counts_df = calculate_statistics_demographics(df=data_dict[demographics[i]], demographic_var=demographics[i], end_date=end_date, event_column='event')
    counts_df
    

## Breakdown by region

,Total Study Period,Within Last Year,Within Last 3 Months
East Midlands,341.0,341.0,137.0
East of England,349.0,349.0,175.0
London,699.0,699.0,276.0
North East,346.0,346.0,151.0
North West,388.0,388.0,167.0
South East,703.0,703.0,329.0
West Midlands,361.0,361.0,145.0
Yorkshire and the Humber,345.0,345.0,150.0


In [30]:
    plot_measures(data_dict[demographics[i]], title=f'Breakdown by {demographics[i]}', column_to_plot='rate_standardised', category=demographics[i], y_label='Standardised Rate per 1000')
    i+=1
    

In [31]:
    display(
    md(f"## Breakdown by {demographics[i]}")
    )
    counts_df = calculate_statistics_demographics(df=data_dict[demographics[i]], demographic_var=demographics[i], end_date=end_date, event_column='event')
    counts_df
    

## Breakdown by imd

,Total Study Period,Within Last Year,Within Last 3 Months
Most deprived,6834.0,6834.0,2919.0
2,7063.0,7063.0,3051.0
3,6932.0,6932.0,2954.0
4,6988.0,6988.0,2994.0
Least deprived,7183.0,7183.0,3082.0


In [32]:
    plot_measures(data_dict[demographics[i]], title=f'Breakdown by {demographics[i]}', column_to_plot='rate_standardised', category=demographics[i], y_label='Standardised Rate per 1000')
    i+=1
    

In [33]:
    display(
    md(f"## Breakdown by {demographics[i]}")
    )
    counts_df = calculate_statistics_demographics(df=data_dict[demographics[i]], demographic_var=demographics[i], end_date=end_date, event_column='event')
    counts_df
    

## Breakdown by ethnicity

,Total Study Period,Within Last Year,Within Last 3 Months
Asian,298,298,145
Black,271,271,112
Mixed,256,256,111
Other,263,263,108
White,298,298,116


In [34]:
    plot_measures(data_dict[demographics[i]], title=f'Breakdown by {demographics[i]}', column_to_plot='rate_standardised', category=demographics[i], y_label='Standardised Rate per 1000')
    i+=1
    

In [35]:
    display(
    md(f"## Breakdown by {demographics[i]}")
    )
    counts_df = calculate_statistics_demographics(df=data_dict[demographics[i]], demographic_var=demographics[i], end_date=end_date, event_column='event')
    counts_df
    

## Breakdown by learning_disability

,Total Study Period,Within Last Year,Within Last 3 Months
0,34640.0,34640.0,14844.0
1,360.0,360.0,156.0


In [36]:
    plot_measures(data_dict[demographics[i]], title=f'Breakdown by {demographics[i]}', column_to_plot='rate_standardised', category=demographics[i], y_label='Standardised Rate per 1000')
    i+=1
    

In [53]:
standard_pop = pd.read_csv('european_standard_population.csv')
standard_pop

,AgeGroup,EuropeanStandardPopulation
0,0-4 years,5000
1,5-9 years,5500
2,10-14 years,5500
3,15-19 years,5500
4,20-24 years,6000
5,25-29 years,6000
6,30-34 years,6500
7,35-39 years,7000
8,40-44 years,7000
9,45-49 years,7000


In [56]:
age_band_grouping_dict = {
            '0-4 years': '0-19',
            '5-9 years': '0-19',
            '10-14 years': '0-19',
            '15-19 years': '0-19',
            '20-24 years': '20-29',
            '25-29 years': '20-29',
            '30-34 years': '30-39',
            '35-39 years': '30-39',
            '40-44 years': '40-49',
            '45-49 years': '40-49',
            '50-54 years': '50-59',
            '55-59 years': '50-59',
            '60-64 years': '60-69',
            '65-69 years': '60-69',
            '70-74 years': '70-79',
            '75-79 years': '70-79',
            '80-84 years': '80+',
            '85-89 years': '80+',
            '90plus years': '80+',
        }

standard_pop.set_index('AgeGroup', inplace=True)
standard_pop = standard_pop.groupby(age_band_grouping_dict, axis=0).sum()


In [64]:
standard_pop = standard_pop.reset_index().rename(columns={'index': 'AgeGroup'})

In [65]:
standard_pop

,level_0,AgeGroup,EuropeanStandardPopulation
0,0,0-19,21500
1,1,20-29,12000
2,2,30-39,13500
3,3,40-49,14000
4,4,50-59,13500
5,5,60-69,11500
6,6,70-79,9000
7,7,80+,5000
